In [7]:
import os
from openslide import OpenSlide, open_slide, deepzoom
import numpy as np
from scipy import linalg
import napari
from skimage.util import img_as_float64
from dask import delayed
import dask.array as da
#os.chdir("../..")

from skimage.measure import label

In [8]:
def getHEDfromRGB(rgb):
    # Conversion matrix for RGB-HED
    rgb_from_hed = np.array([[0.65, 0.70, 0.29],
                            [0.07, 0.99, 0.11],
                            [0.27, 0.57, 0.78]])
    hed_from_rgb = linalg.inv(rgb_from_hed)

    # Modifying _prepare_colorarray and [0,255] -> [0,1]
    rgb = da.asanyarray(rgb) / 255
    # log artifiacts
    da.maximum(rgb, 1E-6, out=rgb)
    # Compensate sum
    log_adjust = da.log(1E-6)
    hed = (da.log(rgb)/log_adjust) @ hed_from_rgb
    da.maximum(hed, 0, out=hed)

    # Do the thresholding
    h_mask = hed[:,:,0]
    h_mask[h_mask >= 0.042] = 1
    h_mask.astype("uint8")
    return da.asanyarray(h_mask)


In [9]:
def svs2dask_label_pyramid(img, gen, level, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False):	
	max_level = gen.level_count - 1 - 2 * level
	n_tiles_x, n_tiles_y = gen.level_tiles[max_level]
	get_tile = lambda level,i,j: np.array(gen.get_tile(level,(i,j))).transpose((1,0,2))
	sample_tile = get_tile(max_level,0,0)
	sample_tile_shape = sample_tile.shape
	dask_get_tile = delayed(get_tile, pure=True)
	arr = (da.concatenate([da.concatenate([da.from_delayed(dask_get_tile(max_level,i,j),sample_tile_shape,np.uint8) for j in range(n_tiles_y)],allow_unknown_chunksizes=allow_unknown_chunksizes,axis=1) for i in range(n_tiles_x )],allow_unknown_chunksizes=allow_unknown_chunksizes))#.transpose([1,0,2]))

	if level == 0:
		hedarr = getHEDfromRGB(arr)
	else:
		# If not zoomed in enough, no mask
		hedarr = da.zeros(shape=arr[:,:,0].shape)

	return hedarr

def svs2dask_array_pyramid(img, gen, level, tile_size=1000, overlap=0, remove_last=True, allow_unknown_chunksizes=False):	
	max_level = gen.level_count - 1 - 2 * level
	n_tiles_x, n_tiles_y = gen.level_tiles[max_level]
	get_tile = lambda level,i,j: np.array(gen.get_tile(level,(i,j))).transpose((1,0,2))
	sample_tile = get_tile(max_level,0,0)
	sample_tile_shape = sample_tile.shape
	dask_get_tile = delayed(get_tile, pure=True)
	arr = (da.concatenate([da.concatenate([da.from_delayed(dask_get_tile(max_level,i,j),sample_tile_shape,np.uint8) for j in range(n_tiles_y)],allow_unknown_chunksizes=allow_unknown_chunksizes,axis=1) for i in range(n_tiles_x )],allow_unknown_chunksizes=allow_unknown_chunksizes))#.transpose([1,0,2]))

	return arr



sm_path = "D:/repos/swe22_final/sm.svs"
img=open_slide(sm_path)
gen=deepzoom.DeepZoomGenerator(img)
lvls = len(open_slide(sm_path).level_dimensions)
viewer = napari.Viewer() 

#imgPy = [svs2dask_array_pyramid(img, gen, level) for level in range(lvls)]

viewer.add_image([svs2dask_array_pyramid(img, gen, level) for level in range(lvls)], contrast_limits=[0,255])
viewer.add_labels([svs2dask_label_pyramid(img, gen, level).astype("uint8") for level in range(lvls)], seed = 0.05)


<Labels layer 'Labels' at 0x1352c35d130>

In [10]:
img=open_slide(sm_path)
gen=deepzoom.DeepZoomGenerator(img, tile_size=1000, overlap=0, limit_bounds=True)
gen.level_dimensions

((1, 1),
 (1, 2),
 (2, 3),
 (4, 5),
 (8, 10),
 (16, 20),
 (32, 39),
 (64, 77),
 (128, 154),
 (255, 308),
 (510, 615),
 (1020, 1229),
 (2040, 2458),
 (4080, 4916),
 (8160, 9832),
 (16320, 19664),
 (32639, 39328))

In [11]:
viewer = napari.Viewer()
#t, c, z, y_slice, x_slice

: 